In [ ]:
import os
import json
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point

import contextily as cx

In [ ]:
#first get alerts with flooding
paths = []
not_found = []
count = 0
base_dir = "data/03-2023"

base_path = base_dir
for day in os.listdir(base_path):
    for hour in os.listdir(os.path.join(base_path, day)):
        for minute in os.listdir(os.path.join(base_path, day, hour)):
            for file_name in os.listdir(os.path.join(base_path, day, hour, minute)):
                path = os.path.join(base_path, day, hour, minute, file_name, f"{file_name}.json")
                kml_path = os.path.join(base_path, day, hour, minute, file_name, f"{file_name}.kml")
                if os.path.exists(path):
                    f = open(path)
                    data = json.load(f)
                    flooded = data["hasFlooding"]

                    if flooded:
                        paths.append(kml_path)
                else:
                    not_found.append(path)
                count += 1

print(f"From {count} alerts, found: {len(paths)} floods, and {len(not_found)} missing json")

In [ ]:
brazil_lower_left = [-28.93576585672869, -70.90469836359233]
brazil_upper_right = [5.696972115053628, -33.84640810546731]
la_map = os.path.join("latam_shp", "vc965bq8111.shp")

not_found = []
alert_in_brazil = []

centroids = []
index = []

for idx, path in enumerate(paths):
    if not os.path.exists(path):
        not_found.append(path)
        continue
    
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file(path, driver='KML')
    aoi_centroid = gdf.loc[gdf["Name"] == "aoi_polygon", "geometry"].representative_point().values[0]
    centroids.append(aoi_centroid)
    index.append(idx)
    lon = int(aoi_centroid.x)
    lat = int(aoi_centroid.y)

    if lon > brazil_lower_left[1] and lon < brazil_upper_right[1]:
        if lat > brazil_lower_left[0] and lat < brazil_upper_right[0]:
            alert_in_brazil.append(path)

gdf = gpd.GeoDataFrame(geometry=centroids)
gdf.head()

In [ ]:
gdf_latam = gpd.read_file(la_map)
fig, ax = plt.subplots(1,1)
gdf_latam.boundary.plot(ax=ax)
gdf.plot(ax=ax, color="red")

In [ ]:
complete_kml = []
incomplete_kml = []

for path in alert_in_brazil:
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file(path, driver='KML')
    if gdf.shape[0] > 1:
        complete_kml.append(path)
    else:
        incomplete_kml.append(path)
    
print(f"From {len(alert_in_brazil)}, {len(complete_kml)} are complete.")